In [3]:
%matplotlib inline


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import utide

print(utide.__version__)

0.3.1


In [2]:
# pip install utide

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 5.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: /global/software/rocky-8.x86_64/python-3.11.6/python-3.11.6/python/3.11.6-gcc/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Look at the data file to see what structure it has.

In [4]:
with open("can1998.dtf") as f:
    lines = f.readlines()

print("".join(lines[:5]))

         0 1998  1  1  0.0000     1.200 0
      3600 1998  1  1  1.0000     1.430 0
      7200 1998  1  1  2.0000     1.730 0
     10800 1998  1  1  3.0000     2.030 0
     14400 1998  1  1  4.0000     2.380 0



It looks like the fields are seconds, year, month, day, hour, elevation, flag.  We need a date parser function to combine the date and time fields into a single value to be used as the datetime index.

In [5]:
names = ["seconds", "year", "month", "day", "hour", "elev", "flag"]

obs = pd.read_csv(
    "can1998.dtf",
    names=names,
    skipinitialspace=True,
    delim_whitespace=True,
    na_values="9.990",
)


date_cols = ["year", "month", "day", "hour"]
index = pd.to_datetime(obs[date_cols])
obs = obs.drop(date_cols, axis=1)
obs.index = index

obs.head(5)

,seconds,elev,flag
1998-01-01 00:00:00,0,1.20,0
1998-01-01 01:00:00,3600,1.43,0
1998-01-01 02:00:00,7200,1.73,0
1998-01-01 03:00:00,10800,2.03,0
1998-01-01 04:00:00,14400,2.38,0


Although there are no elevations marked bad via special value, which should be `nan` after reading the file, the flag value of 2 indicates the values are unreliable, so we will mark them with `nan`, calculate the deviations of the elevations from their mean (stored in a new column called "anomaly"), and then interpolate to fill in the `nan` values in the anomaly.

In [6]:
bad = obs["flag"] == 2
corrected = obs["flag"] == 1

obs.loc[bad, "elev"] = np.nan
obs["anomaly"] = obs["elev"] - obs["elev"].mean()
obs["anomaly"] = obs["anomaly"].interpolate()
print(f"{bad.sum()} points were flagged 'bad' and interpolated")
print(f"{corrected.sum()} points were flagged 'corrected' and left unchanged")

10 points were flagged 'bad' and interpolated
212 points were flagged 'corrected' and left unchanged


Now we can call solve to obtain the coefficients.

In [7]:
coef = utide.solve(
    obs.index,
    obs["anomaly"],
    lat=-25,
    method="ols",
    conf_int="MC",
    verbose=False,
)

The amplitudes and phases from the fit are now in the `coef` data structure (a Bunch), which can be used directly in the `reconstruct` function to generate a hindcast or forecast of the tides at the times specified in the `time` array.

In [ ]:
print(coef.keys())

In [ ]:
tide = utide.reconstruct(obs.index, coef, verbose=False)

The output from the reconstruction is also a Bunch:

In [ ]:
print(tide.keys())

In [ ]:
t = obs.index.to_pydatetime()

fig, (ax0, ax1, ax2) = plt.subplots(figsize=(17, 5), nrows=3, sharey=True, sharex=True)

ax0.plot(t, obs.anomaly, label="Observations", color="C0")
ax1.plot(t, tide.h, label="Prediction", color="C1")
ax2.plot(t, obs.anomaly - tide.h, label="Residual", color="C2")
fig.legend(ncol=3, loc="upper center");